In [1]:
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, Float, String
from sqlalchemy.orm import declarative_base, sessionmaker
import os

Base = declarative_base()

# Read CSV
df = pd.read_csv('G24_Automotive – Dealers  Vehicles with Service Centers_merged.csv', sep=',')

# Create single engine and session
def create_engine_and_session(db_path):
    os.makedirs(os.path.dirname(db_path), exist_ok=True)
    engine = create_engine(f'sqlite:///{db_path}')
    Session = sessionmaker(bind=engine)
    return engine, Session()

# Define models
class Vehicle(Base):
    __tablename__ = 'vehicle'
    vehicle_id = Column(String, primary_key=True)
    model = Column(String)
    make = Column(String)
    price = Column(Float)

class ServiceCenter(Base):
    __tablename__ = 'service_center'
    servicecenter_id = Column(String, primary_key=True)
    service_location = Column(String)
    center_name = Column(String)

class Dealer(Base):
    __tablename__ = 'dealer'
    dealer_id = Column(String, primary_key=True)
    name = Column(String)
    location = Column(String)
    rating = Column(Float)

class Sale(Base):
    __tablename__ = 'sales'
    sale_id = Column(Integer, primary_key=True)
    dealer_id = Column(String)
    vehicle_id = Column(String)
    sale_price = Column(Float)
    sale_date = Column(String)  # Use Date if needed

# Create the unified database
db_path = 'data/automotive.db'
engine, session = create_engine_and_session(db_path)
Base.metadata.create_all(engine)

# Insert data
if {'vehicle_id', 'model', 'make', 'price'}.issubset(df.columns):
    vehicle_df = df[['vehicle_id', 'model', 'make', 'price']].drop_duplicates().dropna().sort_values(by="vehicle_id")
    if not vehicle_df.empty:
        print("Saving Vehicles data...")
        vehicle_df.to_sql('vehicle', engine, if_exists='replace', index=False)

if {'servicecenter_id', 'service_location', 'center_name'}.issubset(df.columns):
    center_df = df[['servicecenter_id', 'service_location', 'center_name']].drop_duplicates().dropna().sort_values(by="servicecenter_id")
    if not center_df.empty:
        print("Saving Service Centers data...")
        center_df.to_sql('service_center', engine, if_exists='replace', index=False)

if {'dealer_id', 'name', 'location', 'rating'}.issubset(df.columns):
    dealer_df = df[['dealer_id', 'name', 'location', 'rating']].drop_duplicates().dropna().sort_values(by="dealer_id")
    if not dealer_df.empty:
        print("Saving Dealers data...")
        dealer_df.to_sql('dealer', engine, if_exists='replace', index=False)

if {'dealer_id', 'vehicle_id', 'sale_date', 'sale_price'}.issubset(df.columns):
    sales_df = df[['dealer_id', 'vehicle_id', 'sale_date', 'sale_price']].dropna().copy()
    sales_df['sale_id'] = range(1, len(sales_df) + 1)
    sales_df = sales_df[['sale_id', 'dealer_id', 'vehicle_id', 'sale_price', 'sale_date']]
    if not sales_df.empty:
        print("Saving Sales data...")
        sales_df.to_sql('sales', engine, if_exists='replace', index=False)

# Close the session
session.close()


Saving Vehicles data...
Saving Service Centers data...
Saving Dealers data...
Saving Sales data...
